## Questions

1. Come up with efficient data structures to represent both types of images: those generated
by the microscope, and those generated by the dye sensor. These need not have the same
representation; the only requirement is that they be compact and take as little storage
space as possible. Explain why you picked the representation you did for each image type,
and if possible estimate how much storage would be taken by the images. What is the
worst-case storage size in bytes for each image representation you chose?

> Answer

Since microscope images only consist of black (blob) and white (background) pixels, we can represent them using a sparse matrix data structure. A sparse matrix is suitable because most of the pixels in the image will be white (background), so we only need to store the coordinates of the black pixels (blob).

One efficient way to represent this is using a dictionary (or hash map) where the keys are the coordinates of the black pixels and the values are either a boolean flag or a fixed value indicating the presence of the blob.

> Storage calculation assuming each coordinate requires 4 bytes,

> Total number of black pixels (assuming worst-case) = 100,000 * 100,000 = 10^10 pixels.

> Total storage = 10^10 pixels * 4 bytes/pixel = 40 GB

For the dye sensor images, we need to efficiently store information about which pixels contain dye. Similar to microscope images, we can use a sparse matrix representation to store only the coordinates of the lit pixels.

Again, a dictionary (hash map) could be used where keys represent the coordinates of lit pixels, and values indicate the presence of dye. The storage rquiremwnts would be similar to that of the microscope images.

Following is an exmaple of the approach. I've used just 100 pixels for the size of image given my local machine's computing power.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random

In [2]:
class SparseImage:
    def __init__(self):
        self.data = defaultdict(bool)  # Using defaultdict for efficient sparse storage

    def add_pixel(self, x, y):
        self.data[(x, y)] = True  # Mark the pixel as part of the image

    def has_pixel(self, x, y):
        return self.data.get((x, y), False)  # Check if the pixel is part of the image

    def total_pixels(self):
        return len(self.data)  # Total number of non-white pixels

In [3]:
# Create a microscope image
microscope_image = SparseImage()

# Assuming we have coordinates of black pixels
black_pixels_coordinates = [(x, y) for x in range(100) for y in range(100)]
for coord in black_pixels_coordinates:
    microscope_image.add_pixel(coord[0], coord[1])

# Create a dye sensor image
dye_sensor_image = SparseImage()

# Assuming we have coordinates of lit pixels
lit_pixels_coordinates = [(x, y) for x in range(100) for y in range(100)]  # Assuming all pixels are lit
for coord in lit_pixels_coordinates:
    dye_sensor_image.add_pixel(coord[0], coord[1])


In [4]:
# Test if a given pixel is in the microscope image or dye sensor image
print("Pixel (4, 4) in microscope image:", microscope_image.has_pixel(4, 4))
print("Pixel (4, 4) in dye sensor image:", dye_sensor_image.has_pixel(4, 4))

# Get total number of non-white pixels in each image
print("Total pixels in microscope image:", microscope_image.total_pixels())
print("Total pixels in dye sensor image:", dye_sensor_image.total_pixels())

Pixel (4, 4) in microscope image: True
Pixel (4, 4) in dye sensor image: True
Total pixels in microscope image: 10000
Total pixels in dye sensor image: 10000


2. Before the researchers give you real images to work with, you would like to test out any
code you write. To this end, you would like to create “fake” simulated images and pretend
they were captured by the microscope and the dye sensor. Using the data structures you
chose in (1) above, write code to create such simulated images. Try and be as realistic in the
generated images as possible.


In [5]:
def generate_microscope_image(size, blob_density):
    microscope_image = SparseImage()
    total_pixels = size * size
    blob_pixels = int(total_pixels * blob_density)

    # Generate coordinates for blob pixels
    blob_coordinates = set()
    while len(blob_coordinates) < blob_pixels:
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        blob_coordinates.add((x, y))

    # Add blob pixels to the microscope image
    for coord in blob_coordinates:
        microscope_image.add_pixel(coord[0], coord[1])

    return microscope_image

In [6]:
def generate_dye_sensor_image(size, dye_density):
    dye_sensor_image = SparseImage()
    total_pixels = size * size
    dye_pixels = int(total_pixels * dye_density)

    # Generate coordinates for dye pixels
    dye_coordinates = set()
    while len(dye_coordinates) < dye_pixels:
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        dye_coordinates.add((x, y))

    # Add dye pixels to the dye sensor image
    for coord in dye_coordinates:
        dye_sensor_image.add_pixel(coord[0], coord[1])

    return dye_sensor_image


In [7]:
# Parameters for simulated images
image_size = 100  
microscope_blob_density = 0.0001  #density of blobs in microscope image
dye_density_within_blob = 0.8  # density of dye within blob in dye sensor image
dye_density_outside_blob = 0.01  # density of dye outside blob in dye sensor image

# Generate simulated microscope image
simulated_microscope_image = generate_microscope_image(image_size, microscope_blob_density)

# Generate simulated dye sensor image
simulated_dye_sensor_image = SparseImage()

# Add dye pixels within blobs
for coord in simulated_microscope_image.data:
    if random.random() < dye_density_within_blob:
        simulated_dye_sensor_image.add_pixel(coord[0], coord[1])

# Add dye pixels outside blobs
total_dye_pixels_outside_blob = int(image_size * image_size * dye_density_outside_blob)
for _ in range(total_dye_pixels_outside_blob):
    x = random.randint(0, image_size - 1)
    y = random.randint(0, image_size - 1)
    if (x, y) not in simulated_microscope_image.data:  # Ensure pixel is not part of blob
        simulated_dye_sensor_image.add_pixel(x, y)

# Testing the generated images
print("Total pixels in simulated microscope image:", simulated_microscope_image.total_pixels())
print("Total pixels in simulated dye sensor image:", simulated_dye_sensor_image.total_pixels())

Total pixels in simulated microscope image: 1
Total pixels in simulated dye sensor image: 100


3. Using the simulated images generated by the code you wrote for (2) above as input, write a
function to compute whether a parasite has cancer or not.


In [8]:
def has_cancer(microscope_image, dye_sensor_image):
    total_blob_pixels = 0
    total_dye_pixels_within_blob = 0

    # Count total blob pixels and dye pixels within blob
    for coord in microscope_image.data:
        total_blob_pixels += 1
        if dye_sensor_image.has_pixel(coord[0], coord[1]):
            total_dye_pixels_within_blob += 1

    # Compute percentage of dye within blob
    percentage_dye_within_blob = (total_dye_pixels_within_blob / total_blob_pixels) * 100

    # Check if percentage exceeds 10%
    if percentage_dye_within_blob > 10:
        return True  # Parasite has cancer
    else:
        return False  # Parasite does not have cancer

# Test the function with simulated images
has_cancer_flag = has_cancer(simulated_microscope_image, simulated_dye_sensor_image)
if has_cancer_flag:
    print("The parasite has cancer.")
else:
    print("The parasite does not have cancer.")


The parasite has cancer.


4. You give your code from (3) to the researchers, who run it and find that it is running too
slowly for their liking. What can you do to improve the execution speed? Write the code to
implement the fastest possible version you can think of for the function in (3).


In [9]:
def has_cancer_fastest(microscope_image, dye_sensor_image):
    # Convert sparse image data to numpy arrays
    microscope_array = np.zeros((image_size, image_size), dtype=bool)
    for coord in microscope_image.data:
        microscope_array[coord[0], coord[1]] = True

    dye_array = np.zeros((image_size, image_size), dtype=bool)
    for coord in dye_sensor_image.data:
        dye_array[coord[0], coord[1]] = True

    # Count total blob pixels
    total_blob_pixels = microscope_image.total_pixels()

    # Count dye pixels within blob using boolean array indexing
    total_dye_pixels_within_blob = np.sum(dye_array & microscope_array)

    # Compute percentage of dye within blob
    percentage_dye_within_blob = (total_dye_pixels_within_blob / total_blob_pixels) * 100

    # Check if percentage exceeds 10%
    if percentage_dye_within_blob > 10:
        return True  # Parasite has cancer
    else:
        return False  # Parasite does not have cancer

# Test the fastest function with simulated images
has_cancer_flag_fastest = has_cancer_fastest(simulated_microscope_image, simulated_dye_sensor_image)
if has_cancer_flag_fastest:
    print("The parasite has cancer.")
else:
    print("The parasite does not have cancer.")


The parasite has cancer.


5. What other compression techniques can you suggest for both types of images (parasite and
dye)? How would they impact runtime? Can you compute actual runtime and storage costs
for typical images (not oversimplified image such as a circle for the parasite, or simple
straight lines or random points for dye) in your code? The measurements should be done on
your computer with an actual image size of 100,000x100,000 pixels (and not a scaled down
version).


In [ ]:
import time
import sys

def rle_compress(image):
    compressed_data = []
    count = 1
    prev_pixel = image[0]
    for pixel in image[1:]:
        if pixel == prev_pixel:
            count += 1
        else:
            compressed_data.extend([prev_pixel, count])
            count = 1
            prev_pixel = pixel
    compressed_data.extend([prev_pixel, count])
    return compressed_data

def rle_decompress(compressed_data):
    decompressed_image = []
    for i in range(0, len(compressed_data), 2):
        pixel = compressed_data[i]
        count = compressed_data[i + 1]
        decompressed_image.extend([pixel] * count)
    return decompressed_image

# Generate a typical parasite image (random binary image)
typical_parasite_image = np.random.randint(0, 2, (100000, 100000), dtype=bool)

# Measure storage space before compression
original_storage = sys.getsizeof(typical_parasite_image)

# Measure compression time
start_time = time.time()
compressed_data = rle_compress(typical_parasite_image.flatten())
compression_time = time.time() - start_time

# Measure storage space after compression
compressed_storage = sys.getsizeof(compressed_data)

# Measure decompression time
start_time = time.time()
decompressed_image = rle_decompress(compressed_data)
decompression_time = time.time() - start_time

# Output results
print("Original storage:", original_storage, "bytes")
print("Compressed storage:", compressed_storage, "bytes")
print("Compression time:", compression_time, "seconds")
print("Decompression time:", decompression_time, "seconds")


6. Describes what tools you used to solve the challenge, particularly any LLM techniques.

To solve the challenge, I used Python along with libraries such as NumPy for computations. I employed sparse matrix data structures to efficiently represent microscope and dye sensor images due to their suitability for handling large images with a significant amount of empty space. Algorithm optimization techniques were applied to improve runtime performance, minimizing redundant iterations and utilizing efficient data structures and operations. Additionally, I utilized Run-Length Encoding (RLE) compression to reduce storage space for simulated images, a simple yet effective compression method. The simulation of images was achieved using random data generation techniques. While I didn't directly utilize Large Language Model (LLM) techniques, my responses were informed by the extensive knowledge ingrained within the model, allowing me to propose effective solutions to the given problem